<a href="https://colab.research.google.com/github/NamelessOgya/test-cpu-llm/blob/main/sample_nb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 変数

In [ ]:
RESULT_DIR   = '/content/drive/MyDrive/tmp'#結果を格納するディレクトリ
MOUNT_GDRIVE = True # google driveをマウントするか
REPEAT  = 2 #同じプロンプトで何回繰り返し出力するか

# 準備

In [ ]:
import csv

In [ ]:
from datetime import datetime, timezone, timedelta

# JST（UTC+9）タイムゾーンを設定
jst = timezone(timedelta(hours=9))

# 現在のJSTの日時を取得
current_time_jst = datetime.now(jst)

# 日付と時間を文字列で取得
date_str = current_time_jst.strftime("%Y%m%d%H%M%S")

In [ ]:
date_str

'20241103201045'

In [ ]:
%cd /content

if MOUNT_GDRIVE:
  from google.colab import drive
  drive.mount('/content/drive')
  !mkdir drive/MyDrive/tmp

/content
Mounted at /content/drive
mkdir: cannot create directory ‘drive/MyDrive/tmp’: File exists


In [ ]:
from google.colab import userdata

## git clone

In [ ]:
!git clone https://github.com/NamelessOgya/test-cpu-llm.git

Cloning into 'test-cpu-llm'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 101 (delta 38), reused 70 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (101/101), 21.27 KiB | 622.00 KiB/s, done.
Resolving deltas: 100% (38/38), done.


# content

In [ ]:
# ##クラスデバッグ用の、ダミーのLLMクラス
# package_name = "dummy_llm_cpp"
# repo_id = "hoge"
# filename = "huga"

In [ ]:
package_name = "llama_cpp"
repo_id = "mmnga/ELYZA-japanese-Llama-2-7b-fast-instruct-gguf"
filename = "ELYZA-japanese-Llama-2-7b-fast-instruct-q4_K_M.gguf"

In [ ]:
# package_name = "bitnet_cpp"
# repo_id = "HF1BitLLM/Llama3-8B-1.58-100B-tokens"
# filename = "i2_s"

In [ ]:
# package_name = "ollama"
# repo_id = "mmnga/ELYZA-japanese-Llama-2-7b-fast-instruct-gguf"
# filename = "*q8_0.gguf"

# # ollamaには量子化の機能がない

## install / import package

In [ ]:
%cd test-cpu-llm

/content/test-cpu-llm


In [ ]:
print(f"package: {package_name}")

!sed -i 's/\r$//' ./src/{package_name}/install.sh
!chmod +x ./src/{package_name}/install.sh
!./src/{package_name}/install.sh

package: llama_cpp
Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 32.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.0 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.1-cp310-cp310-linux_x86_64.whl size=3482754 sha256=6d95647ff17a705ea42be1c00c1dd9ecfea4e65e29a3dd1378de3320debda5d8
  Stored in directory: /root/.cache/pip/wheels/f8/b0/a2/f47d952aec7ab061b9e2a345e23a1e1e137beb7891259e3d0c
Successfully built llama-cpp-python


In [ ]:
import sys
sys.path.append(f'./src/{package_name}')
sys.path.append(f'./src/common')

In [ ]:
from llm_module import LLMModule

# LLMModuleのインスタンス化

※ bitnet.cppにおいてはとても時間かかります。30GB程度のRAMが必要です。ログが標準出力に出力されません

In [ ]:
llm = LLMModule(repo_id, filename)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


(…)ese-Llama-2-7b-fast-instruct-q4_K_M.gguf:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

# 実験開始

## 実験メニューの読み込み

In [ ]:
# 記録用csvを作成
result_filename = RESULT_DIR + "/" + date_str + ".csv"


# ヘッダーを追加
with open(result_filename, mode="a", newline="", encoding="utf-8") as file:
  writer = csv.writer(file)
  writer.writerow(["package_name", "repo_id", "filename", "職種", "業種","業務内容","スキル", "自己PR", "結果", "時間", "メモリ"])

In [ ]:
with open("/content/test-cpu-llm/prompt/pattern.txt", "r", encoding="utf-8") as file:
  pattern = file.read()

In [ ]:
with open("/content/test-cpu-llm/prompt/prompt.txt", "r", encoding="utf-8") as file:
  content_base = file.read()

In [ ]:

with open("/content/test-cpu-llm/prompt/output_format.txt", "r", encoding="utf-8") as file:
  output_format = file.read()

In [ ]:
for repeat in range(REPEAT):
  print( "===============================")
  print(f"=== repeat {repeat} start!! ===")
  print( "===============================")

  for n, i in enumerate(pattern.split("\n")):
    if (n % 10 == 0) | (n == 1):
      print(f"infer for {n} sample..")

    sample = content_base.format(
        shokushu = i.split(",")[0],
        gyoushu  = i.split(",")[1],
        naiyou   = i.split(",")[2],
        skill    = i.split(",")[3],
        self_pr  = i.split(",")[4]
    )
    input = sample + "出力の際は必ず以下のフォーマットを守ってください。これから列挙するkeyに対して値の要件を満たすようなテキストを出力してください。\n" + output_format
    r = llm.infer_with_metrics(input)

    r["package_name"] = package_name
    r["repo_id"]      = repo_id
    r["filename"]     = filename

    with open(result_filename, mode="a", newline="", encoding="utf-8") as file:
      writer = csv.writer(file)
      writer.writerow(
        [
          r["package_name"],
          r["repo_id"],
          r["filename"],

          i.split(",")[0],
          i.split(",")[1],
          i.split(",")[2],
          i.split(",")[3],
          i.split(",")[4],

          r["result"],
          "{:.2f}".format(r["time"]),
          "{:.2f}".format(r["memory"]),
        ]
      )



=== repeat 0 start!! ===
infer for 0 sample..
infer for 1 sample..
infer for 10 sample..
infer for 20 sample..
infer for 30 sample..
infer for 40 sample..
=== repeat 1 start!! ===
infer for 0 sample..
infer for 1 sample..
infer for 10 sample..
infer for 20 sample..
infer for 30 sample..
infer for 40 sample..


In [ ]:
input

'現在転職を考えています。\nこれから記載する経歴を参考にして、私のための職務経歴書を作成してください。  \n  \n職種: 法人営業\n業種:  金融\n業務内容: 法人顧客への資産管理提案、商談対応\nスキル:  営業経験\n自己PR:  対話力、\n出力の際は必ず以下のフォーマットを守ってください。これから列挙するkeyに対して値の要件を満たすようなテキストを出力してください。\nkey; 概要\nvalue: これまで行ってきた職務の概要を記載してください。\n\nkey: 業務内容\nvalue: これまで行ってきた業務の内容を箇条書きで記載\n\nkey: 成果\nvalue: 業務の結果得られた成果を記載\n\nkey: 活かしたスキル\nvalue: 職務遂行の際に活かしたスキルを記載\n\nkey: 自己PR\nvalue: 自己PRを記載。各項目に関して項目と具体的内容を記載すること'

In [ ]:
r

{'time': 18.357044458389282,
 'memory': 5.338977813720703,
 'result': '{"status": "OK", "response": "転職相談を受け付けます。現在の職務内容、転職希望理由、希望する業界や職種、希望する転職先の規模、希望する転職の期限などを教えていただけますと、より具体的な提案が可能です。"}',
 'package_name': 'llama_cpp',
 'repo_id': 'elyza/Llama-3-ELYZA-JP-8B-GGUF',
 'filename': 'Llama-3-ELYZA-JP-8B-q4_k_m.gguf'}

In [ ]:
r

{'time': 2.3332202434539795,
 'memory': 5.338726043701172,
 'result': '{"status": "準備完了"}',
 'package_name': 'llama_cpp',
 'repo_id': 'elyza/Llama-3-ELYZA-JP-8B-GGUF',
 'filename': 'Llama-3-ELYZA-JP-8B-q4_k_m.gguf'}

# develop

In [ ]:
from llama_cpp  import Llama

In [ ]:
sample = '''

現在転職を考えています。
これから記載する経歴を参考にして、私のための職務経歴書を作成してください。

職種: 営業
業種: インターネットサービス
スキル: ビジネスレベル英語
自己PR: 粘り強さ、体力

'''

In [ ]:
llm = Llama.from_pretrained(
    repo_id=repo_id,
    filename=filename,
    verbose=False,
    n_ctx = 4096
)

In [ ]:
sample_format = '''
key; 概要
value: これまで行ってきた職務の概要を記載してください。

key: 業務内容
value: これまで行ってきた業務の内容を箇条書きで記載

key: 成果
value: 業務の結果得られた成果を記載

key: 活かしたスキル
value: 職務遂行の際に活かしたスキルを記載

key: 自己PR
value: 自己PRを記載。各項目に関して項目と具体的内容を記載すること

'''

In [ ]:
input = sample + "出力の際は必ず以下のフォーマットを守ってください。これから列挙するkeyに対して値の要件を満たすようなテキストを出力してください。\n" + sample_format

In [ ]:
input

'\n\n現在転職を考えています。\nこれから記載する経歴を参考にして、私のための職務経歴書を作成してください。\n\n職種: 営業\n業種: インターネットサービス\nスキル: ビジネスレベル英語\n自己PR: 粘り強さ、体力\n\n出力の際は必ず以下のフォーマットを守ってください。これから列挙するkeyに対して値の要件を満たすようなテキストを出力してください。\n\nkey; 概要\nvalue: これまで行ってきた職務の概要を記載してください。\n\nkey: 業務内容\nvalue: これまで行ってきた業務の内容を箇条書きで記載\n\nkey: 成果\nvalue: 業務の結果得られた成果を記載\n\nkey: 活かしたスキル\nvalue: 職務遂行の際に活かしたスキルを記載\n\nkey: 自己PR\nvalue: 自己PRを記載。各項目に関して項目と具体的内容を記載すること\n\n'

In [ ]:
out = llm.create_chat_completion(
    messages=[
        {
            "role": "system",
            "content": "あなたは優秀なキャリアアドバイザーです。転職したいクライアントから相談を受け、クライアントにとって最適な提案を行うことでクライアントを支援します。結果は必ず日本語で出力するルールになっています。出力はjsonフォーマットでお願いします",
        },
        {"role": "user", "content": input},
    ],
    response_format={
        "type": "json_object",
    },
    temperature=0.7,
    max_tokens=3000
)

In [ ]:
out["choices"][0]["message"]["content"]

'{"概要": "インターネットサービス企業で営業として勤務してきた経歴です。粘り強さと体力が私の強みです。","business": ["インターネットサービスの営業として、顧客開拓、営業戦略の立案、提案型営業を行いました。","クライアントのニーズに合ったソリューションを提案し、契約を獲得することを目標に、営業活動を展開していました。","新規顧客獲得、既存顧客の深耕、リードの追客を通して、目標の達成を目指してきました。"],"achievement": ["新規顧客獲得の達成率は30%以上を維持し、既存顧客との関係を深化させることで、リピートビジネスや紹介を獲得することができました。","粘り強く交渉し、困難な案件を成功に導くことができたため、信頼を得ることができました。","結果として、目標の達成はもちろん、顧客との信頼関係を構築することができました。"],"skill": ["ビジネスレベルの英語を使用して、海外の顧客やパートナーとの交渉やコミュニケーションを取ることができました。","粘り強さと体力を活かして、困難な交渉や営業活動を通して成果を得ることができました。"],"自己PR": ["私は粘り強い性格と体力で、困難な案件を成功に導くことができます。粘り強く交渉し、顧客のニーズを理解することで、信頼を得ることができます。インターネットサービス業界での経験を活かして、顧客のニーズに合ったソリューションを提案することができます。"]}'

# gitへのpush

In [ ]:
%cd /content/test-cpu-llm
# !git pull --ff-only
!git status

/content/test-cpu-llm
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   prompt/pattern.txt
	modified:   prompt/prompt.txt
	modified:   src/llama_cpp/llm_module.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	prompt/output_format.txt

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git add -n .

add 'prompt/pattern.txt'
add 'prompt/prompt.txt'
add 'src/llama_cpp/llm_module.py'
add 'prompt/output_format.txt'


In [ ]:
!git add --all .

In [ ]:
!git config --global user.email "hakuginnnohana@gmail.com"
!git config --global user.name "NamelessOgya"

In [ ]:
!git commit -m"enhance prompt. implements chat inference"

[main 38fb8fa] enhance prompt. implements chat inference
 4 files changed, 84 insertions(+), 66 deletions(-)
 create mode 100644 prompt/output_format.txt
 rewrite prompt/pattern.txt (95%)


In [ ]:
!git branch

* main


In [ ]:
push_url_with_pat = "https://" + userdata.get('pat') + "@github.com/"+ userdata.get('git_user_name') + "/test-cpu-llm.git"
!git remote set-url origin {push_url_with_pat}

In [ ]:
# !git remote -v

In [ ]:
!git push origin

Enumerating objects: 16, done.
Counting objects: 100% (16/16), done.
Delta compression using up to 8 threads
Compressing objects: 100% (9/9), done.
Writing objects: 100% (9/9), 2.81 KiB | 2.81 MiB/s, done.
Total 9 (delta 4), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (4/4), completed with 4 local objects.
To https://github.com/NamelessOgya/test-cpu-llm.git
   4cf6084..38fb8fa  main -> main


In [ ]:
!git fetch origin

In [ ]:
!git checkout --theirs sample_nb.ipynb

Updated 1 path from the index


In [ ]:
!git add sample_nb.ipynb

In [ ]:
!git rebase --continue

Successfully rebased and updated refs/heads/main.
